Multiple imputation del test anterior

In [1]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#### Tornem a carregar data pel primer nivell

In [2]:
odor = pd.read_json('Data/odor_prop.json', lines=False, orient='records')
odor=odor[odor['level']=='first']
odor.reset_index(drop=True, inplace=True)
mol=odor['cid'].value_counts()
mols=mol.index[mol>1]
groups=[]
for i in range(len(odor)):
    for mol in mols:
        if odor['cid'][i] == mol:
            g=np.max(odor[odor['cid']==mol]['label'])
            groups.append([mol, g])
            ls_drop=[]
for mol in groups:
    for i in range(len(odor)):
        if (odor['cid'][i] == mol[0]):
            if odor['label'][i]!=mol[1]:
                ls_drop.append(i)
odor=odor.drop(ls_drop)
odor.reset_index(drop=True, inplace=True)
odor = odor.sort_values(by='cid')
odor.reset_index(inplace=True)
ls_repeat=[]
for i in range(len(odor)-1):
    if (odor['cid'][i]==odor['cid'][i+1]):
        ls_repeat.append(i)
odor=odor.drop(ls_repeat)

#### Canviem string categoric a int

In [3]:
len(odor['Subclass'].unique()), len(odor['Superclass'].unique()), len(odor['Class'].unique()), len(odor['Direct Parent'].unique())

(69, 10, 65, 135)

In [4]:
odor.replace(to_replace=odor['Subclass'].unique(), 
                   value=np.arange(0,69), inplace=True)
odor.replace(to_replace=odor['Superclass'].unique(), 
                   value=np.arange(0,10), inplace=True)
odor.replace(to_replace=odor['Class'].unique(), 
                   value=np.arange(0,65), inplace=True)
odor.replace(to_replace=odor['Direct Parent'].unique(), 
                   value=np.arange(0,113), inplace=True)

In [7]:
ind_X=['TPSA', 'MolLogP',
       'MolWt', 'ExactMolWt', 'MolMR', 'NumHAcceptors', 'NumHDonors',
       'NumRotatableBonds', 'NumAromaticRings', 'NumSaturatedRings',
       'NumAliphaticRings', 'NumAromaticHeterocycles',
       'NumSaturatedHeterocycles', 'NumAliphaticHeterocycles',
       'NumAromaticCarbocycles', 'NumSaturatedCarbocycles',
       'NumAliphaticCarbocycles', 'NumHeteroatoms', 'NumValenceElectrons',
       'RingCount', 'FractionCSP3', 'LabuteASA', 'BalabanJ', 'BertzCT', 'Ipc',
       'HallKierAlpha', 'Kappa1', 'Kappa2', 'Kappa3', 'Chi0', 'Chi1', 'Chi0n',
       'Chi1n', 'Chi2n', 'Chi3n', 'Chi4n', 'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v',
       'Chi4v', 'HeavyAtom', 'Gasteiger', 'Eccentricity',
       'InertialShapeFactor', 'RadiusOfGyration', 'Asphericity',
       'SpherocityIndex', 'PMI1', 'PMI2', 'PMI3', 'NPR1', 'NPR2',
       'Superclass', 'Class', 'Subclass', 'Direct Parent'
      ]

#### Fem multiple imputation 

In [11]:
odor_aux = odor[ind_X]

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(odor_aux)

IterativeImputer(random_state=0)

In [13]:
odor_aux=imp_mean.transform(odor_aux)

In [15]:
odor_imput = pd.DataFrame(odor_aux, columns=ind_X)

In [18]:
odor_imput

,TPSA,MolLogP,MolWt,ExactMolWt,MolMR,NumHAcceptors,NumHDonors,NumRotatableBonds,NumAromaticRings,NumSaturatedRings,...,SpherocityIndex,PMI1,PMI2,PMI3,NPR1,NPR2,Superclass,Class,Subclass,Direct Parent
0,54.37,0.29610,116.116,116.047344,27.4808,2.0,63.0,2.0,0.0,0.0,...,0.246056,148.857521,317.924313,429.659337,0.346455,0.739945,0.0,0.0,0.0,0.0
1,54.37,0.05010,102.089,102.031694,22.9338,2.0,63.0,2.0,0.0,0.0,...,0.148872,93.924062,284.315371,371.385266,0.252902,0.765554,0.0,0.0,0.0,0.0
2,54.37,0.68620,130.143,130.062994,32.0978,2.0,63.0,3.0,0.0,0.0,...,0.329791,171.302808,497.850680,600.987076,0.285036,0.828388,0.0,0.0,0.0,0.0
3,77.76,0.79600,154.121,154.026609,36.7309,3.0,3.0,63.0,63.0,0.0,...,0.001511,203.865902,733.138397,936.836062,0.217611,0.782569,0.0,0.0,0.0,0.0
4,37.30,1.70380,150.177,150.068080,42.3988,63.0,63.0,3.0,63.0,0.0,...,0.367010,244.806354,699.448694,766.181173,0.319515,0.912902,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,59.67,1.77874,184.191,184.073559,45.8033,6.0,63.0,2.0,63.0,0.0,...,0.113703,357.604196,954.993714,1268.129836,0.281993,0.753073,0.0,0.0,0.0,0.0
3384,97.66,1.89840,289.392,289.202048,76.4176,6.0,2.0,23.0,0.0,0.0,...,0.244771,442.417189,3027.322660,3061.794054,0.144496,0.988741,0.0,0.0,0.0,0.0
3385,17.07,3.73810,194.318,194.167065,60.1770,63.0,0.0,3.0,0.0,63.0,...,0.348291,409.942742,1195.021208,1366.771194,0.299935,0.874339,0.0,0.0,0.0,0.0
3386,20.23,3.92000,222.372,222.198365,68.2268,63.0,63.0,63.0,0.0,63.0,...,0.497367,595.679021,1176.093774,1400.878383,0.425218,0.839540,0.0,0.0,0.0,0.0


In [25]:
odor_imput

,TPSA,MolLogP,MolWt,ExactMolWt,MolMR,NumHAcceptors,NumHDonors,NumRotatableBonds,NumAromaticRings,NumSaturatedRings,...,SpherocityIndex,PMI1,PMI2,PMI3,NPR1,NPR2,Superclass,Class,Subclass,Direct Parent
0,54.37,0.29610,116.116,116.047344,27.4808,2.0,63.0,2.0,0.0,0.0,...,0.246056,148.857521,317.924313,429.659337,0.346455,0.739945,0.0,0.0,0.0,0.0
1,54.37,0.05010,102.089,102.031694,22.9338,2.0,63.0,2.0,0.0,0.0,...,0.148872,93.924062,284.315371,371.385266,0.252902,0.765554,0.0,0.0,0.0,0.0
2,54.37,0.68620,130.143,130.062994,32.0978,2.0,63.0,3.0,0.0,0.0,...,0.329791,171.302808,497.850680,600.987076,0.285036,0.828388,0.0,0.0,0.0,0.0
3,77.76,0.79600,154.121,154.026609,36.7309,3.0,3.0,63.0,63.0,0.0,...,0.001511,203.865902,733.138397,936.836062,0.217611,0.782569,0.0,0.0,0.0,0.0
4,37.30,1.70380,150.177,150.068080,42.3988,63.0,63.0,3.0,63.0,0.0,...,0.367010,244.806354,699.448694,766.181173,0.319515,0.912902,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,59.67,1.77874,184.191,184.073559,45.8033,6.0,63.0,2.0,63.0,0.0,...,0.113703,357.604196,954.993714,1268.129836,0.281993,0.753073,0.0,0.0,0.0,0.0
3384,97.66,1.89840,289.392,289.202048,76.4176,6.0,2.0,23.0,0.0,0.0,...,0.244771,442.417189,3027.322660,3061.794054,0.144496,0.988741,0.0,0.0,0.0,0.0
3385,17.07,3.73810,194.318,194.167065,60.1770,63.0,0.0,3.0,0.0,63.0,...,0.348291,409.942742,1195.021208,1366.771194,0.299935,0.874339,0.0,0.0,0.0,0.0
3386,20.23,3.92000,222.372,222.198365,68.2268,63.0,63.0,63.0,0.0,63.0,...,0.497367,595.679021,1176.093774,1400.878383,0.425218,0.839540,0.0,0.0,0.0,0.0


In [31]:
ind_X=odor.columns[6:59]
X = odor_imput
y = odor['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 

In [34]:
classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LinearDiscriminantAnalysis(),
    make_pipeline(StandardScaler(), SVC(gamma='auto')),
    svm.SVC()
    ]

# Logging for Visual Comparison


for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
      
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 33.3333%
DecisionTreeClassifier
****Results****
Accuracy: 33.7802%
RandomForestClassifier
****Results****
Accuracy: 45.3977%
LinearDiscriminantAnalysis
****Results****
Accuracy: 46.6488%
Pipeline
****Results****
Accuracy: 44.6828%
SVC
****Results****
Accuracy: 32.7971%
